In [54]:
import sys
sys.path.insert(0, "/Users/kaushik/MyStuff/Workspace/NEU/DS5500/Project/DS5500_CapstoneProject")

In [55]:
from src.config import cfg
from src.data import prepOPPCorpus
from src.data import preprocess

from src.models import CNN
import src.models

from src.main import main
from src.main import driver

from src.utils import gen
from src.utils import metrics

import itertools
import numpy as np
import pandas as pd

# Process

In [56]:
gen.setSeeds(seed=2021)

In [57]:
df = gen.loadDataset(cfg)
df

🟢 Dataset loaded!


,segment_text,category
0,Privacy Policy Sci-News.com is committed to ...,"[Policy Change, Introductory/Generic]"
1,Information that Sci-News.com May Collect Onli...,[First Party Collection/Use]
2,"- if you contact us, we may keep a record of t...","[First Party Collection/Use, Data Retention]"
3,- details of your visits to our site including...,[First Party Collection/Use]
4,Sci-News.com does not knowingly collect or sol...,"[International and Specific Audiences, Privacy..."
...,...,...
3771,Information Collected at Mohegan Sun In addi...,"[First Party Collection/Use, Third Party Shari..."
3772,Management of User Information Certain infor...,"[User Access, Edit and Deletion, Privacy conta..."
3773,Special Note About Children This site is not...,"[First Party Collection/Use, International and..."
3774,If You Have a Question If you have a questio...,[Privacy contact information]


In [58]:
num_samples = len(df)
num_samples

3776

In [59]:
df.segment_text = df.segment_text.apply(preprocess.cleanText, lower=True, stem=False)
df.segment_text

0       privacy policy sci news com is committed to pr...
1       information that sci news com may collect onli...
2       if you contact us we may keep a record of that...
3       details of your visits to our site including b...
4       sci news com does not knowingly collect or sol...
                              ...                        
3771    information collected at mohegan sun in additi...
3772    management of user information certain informa...
3773    special note about children this site is not i...
3774    if you have a question if you have a question ...
3775    you may also contact us by mail using the addr...
Name: segment_text, Length: 3776, dtype: object

In [60]:
cats = df.category
label_encoder = preprocess.LabelEncoder()
label_encoder.fit(cats)
y = label_encoder.encode(cats)
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [61]:
cats_list = list(itertools.chain.from_iterable(cats.values))
counts = np.bincount([label_encoder.class_to_index[cat_] for cat_ in cats_list])
cat_weights = {i: 1.0 / count for i, count in enumerate(counts)}
cat_weights

{0: 0.00641025641025641,
 1: 0.0026666666666666666,
 2: 0.03125,
 3: 0.0006583278472679394,
 4: 0.0028328611898017,
 5: 0.0012300123001230013,
 6: 0.005208333333333333,
 7: 0.001557632398753894,
 8: 0.0030959752321981426,
 9: 0.0008431703204047217,
 10: 0.004347826086956522,
 11: 0.0015822784810126582}

In [62]:
gen.setSeeds(seed=2021)

In [63]:
train_size = 0.7

In [64]:
X = df.segment_text.to_numpy()

In [65]:
X_train, X_, y_train, y_ = preprocess.train_test_split_multilabel(
    X=X, y=y, train_size=train_size
)
X_val, X_test, y_val, y_test = preprocess.train_test_split_multilabel(X=X_, y=y_, train_size=0.5)
test_df = pd.DataFrame({"segment_text": X_test, "category": label_encoder.decode(y_test)})

In [66]:
print("Successfully split the dataset into {:g}% train, {:g}% val and {:g}% test!".format((train_size)*100, (1-train_size)/2*100, (1-train_size)/2*100))
print("Number of unique segments in total: {}".format(X.shape[0]))
metrics.splitStatistics(splitlist=[X_train, X_val, X_test, y_train, y_val, y_test])

Successfully split the dataset into 70% train, 15% val and 15% test!
Number of unique segments in total: 3776
TRAIN SET
Number of unique segments: 2645
Percentage of segments containing each of the following categories:
                                      Counts Percentage
Data Retention                           109      2.41%
Data Security                            262       5.8%
Do Not Track                              22      0.49%
First Party Collection/Use              1063     23.55%
International and Specific Audiences     247      5.47%
Introductory/Generic                     569     12.61%
Policy Change                            134      2.97%
Practice not covered                     449      9.95%
Privacy contact information              226      5.01%
Third Party Sharing/Collection           830     18.39%
User Access, Edit and Deletion           161      3.57%
User Choice/Control                      442      9.79%
----------------------------------------------------

In [67]:
char_level = False

In [50]:
tokenizer = preprocess.Tokenizer(char_level=char_level)

tokenizer.fit_on_texts(texts=X_train)

In [68]:
tokenizer = preprocess.Tokenizer(char_level=char_level)

tokenizer.fit_on_texts(texts=X)

In [69]:
X_train = np.array(tokenizer.texts_to_sequences(X_train), dtype=object)
X_val = np.array(tokenizer.texts_to_sequences(X_val), dtype=object)
X_test = np.array(tokenizer.texts_to_sequences(X_test), dtype=object)

In [72]:
c = 0
for lst in X_train:
    if 1 in lst:
        c+=1
c

110